In [1]:
import pandas as pd
import plotly.express as px

In [2]:
#Reading data
stocks_br = pd.read_excel('./ações.xlsx')
avg_daily_liquidity = pd.read_excel('./liquidez.xlsx')

In [3]:
#Knowing the data
print(stocks_br.shape)
print('\n')
stocks_br.head()

(205, 6)




,codigo,setor,preço 23.03,preço 27.11,ibov,percentual
0,BEEF3,consumo não cíclico,7.99,10.13,1,0.130
1,COGN3,consumo cíclico,4.55,4.89,1,0.446
2,DTEX3,materiais básicos,8.24,19.21,0,0.000
3,EQTL3,utilidade pública,16.08,22.14,1,1.104
4,HAPV3,saúde,7.68,14.35,1,0.768


In [4]:
#Verifying that the column types are correct:
stocks_br.dtypes

codigo          object
setor           object
preço 23.03    float64
preço 27.11    float64
ibov             int64
percentual     float64
dtype: object

In [5]:
#All types are correct
#Checking for missing values:
stocks_br.isna().sum()

codigo         0
setor          0
preço 23.03    0
preço 27.11    0
ibov           0
percentual     0
dtype: int64

In [6]:
#Knowing the data
print(avg_daily_liquidity.shape)
print('\n')
avg_daily_liquidity.head()

(234, 2)




,codigo,liquidez media diaria
0,AALR3,4.223010e+06
1,ABCB4,1.064114e+07
2,ABEV3,4.242351e+08
3,AERI3,2.461832e+07
4,AGRO3,4.441010e+06


In [7]:
#Verifying that the column types are correct:
avg_daily_liquidity.dtypes

codigo                    object
liquidez media diaria    float64
dtype: object

In [8]:
#All types are correct
#Checking for missing values:
avg_daily_liquidity.isna().sum()

codigo                   0
liquidez media diaria    0
dtype: int64

In [9]:
stocks_br = pd.merge(stocks_br, avg_daily_liquidity, how='left', on=['codigo'])
stocks_br.shape

(205, 7)

In [18]:
#All columns are correct
#Starting data exploration:

#Who are the 10 companies with the highest percentage on the IBOVESPA?
top_10_highest_comp = stocks_br.sort_values(by=['percentual'], ascending=False).head(10)
fig1 = px.bar(top_10_highest_comp, x='codigo', y='percentual')
fig1.update_layout(
    title_text='',
    yaxis=dict(
        title='Percentage (%)'
    ),
    xaxis=dict(
        title='Company codes'
    ))
fig1.show()
fig1.write_image("./fig1.png")

In [19]:
#Which sectors have the highest percentage on the IBOVESPA?
sectors = stocks_br[stocks_br.ibov == 1]
sectors = sectors.groupby(['setor']).sum().reset_index().sort_values(by=['percentual'], ascending=False)

fig2 = px.bar(sectors, x='setor', y='percentual')
fig2.update_layout(
    title_text='',
    yaxis=dict(
        title='Percentage (%)'
    ),
    xaxis=dict(
        title='Sectors'
    ))
fig2.show()
fig2.write_image("./fig2.png")

In [12]:
#Getting stock performance
stocks_br['performance'] = round(((stocks_br['preço 27.11'] / stocks_br['preço 23.03']) - 1) * 100, 2)
stocks_br

,codigo,setor,preço 23.03,preço 27.11,ibov,percentual,liquidez media diaria,performance
0,BEEF3,consumo não cíclico,7.99,10.13,1,0.130,8.447316e+07,26.78
1,COGN3,consumo cíclico,4.55,4.89,1,0.446,2.801033e+08,7.47
2,DTEX3,materiais básicos,8.24,19.21,0,0.000,6.414306e+07,133.13
3,EQTL3,utilidade pública,16.08,22.14,1,1.104,1.549354e+08,37.69
4,HAPV3,saúde,7.68,14.35,1,0.768,1.255556e+08,86.85
...,...,...,...,...,...,...,...,...
200,MMXM3,materiais básicos,1.40,17.83,0,0.000,2.261461e+07,1173.57
201,OIBR3,comunicações,0.44,2.05,0,0.000,1.936553e+08,365.91
202,CPFE3,utilidade pública,22.21,30.60,1,0.283,7.667341e+07,37.78
203,CPLE3,utilidade pública,47.00,64.73,0,0.000,1.239709e+07,37.72


In [20]:
#What are the 10 stocks that had the highest performance in the period?

top_10_highest_performance = stocks_br.sort_values(by=['performance'], ascending=False).head(10)
fig3 = px.bar(top_10_highest_performance, x='codigo', y='performance')
fig3.update_layout(
    yaxis=dict(
        title='Percentage (%)'
    ),
    xaxis=dict(
        title='Company codes'
    ))
fig3.show()
fig3.write_image("./fig3.png")



In [21]:
#What are the 10 stocks that had the worst performance in the period?

top_10_worst_performance = stocks_br.sort_values(by=['performance'], ascending=True).head(10)
fig4 = px.bar(top_10_worst_performance, x='codigo', y='performance')
fig4.update_layout(
    yaxis=dict(
        title='Percentage (%)'
    ),
    xaxis=dict(
        title='Company codes'
    ))
fig4.show()
fig4.write_image("./fig4.png")

In [15]:
#Number of companies that make up the ibovespa index
print(stocks_br[stocks_br.ibov == 1].shape[0])

#Number of companies that are not part of the ibovespa index
print(stocks_br[stocks_br.ibov == 0].shape[0])

76
129


In [23]:
#As 76 shares make up the IBOV, and to make a fair comparison, 
#we will take the 76 shares with the highest liquidity that do not make up the ibov.

only_ibov = stocks_br[stocks_br.ibov == 1]
only_others = stocks_br[stocks_br.ibov == 0].sort_values(by=['liquidez media diaria'], ascending=False).head(only_ibov.shape[0])

print(only_others.shape)


stocks = pd.concat([only_ibov, only_others])
stocks['setor'].replace('Bens industriais', 'bens industriais', inplace=True)
print(stocks.shape)

(76, 8)
(152, 8)


In [24]:
#Do the stocks that make up the ibovespa index perform better?
#Performance of actions by sector and group

fig5 = px.box(stocks, x="setor", y="performance", color="ibov")
fig5.update_traces(quartilemethod="linear") #"exclusive" or "inclusive", or "linear" by default
fig5.update_layout(
    title_text='Performance of actions by sector and group',
    yaxis=dict(
        title='Percentage (%)'
    ),
    xaxis=dict(
        title='Sectors'
    ))
fig5.show()
fig5.write_image("./fig5.png")